Oskar Lundqvist; osauli-0@student.ltu.se

Filip Renberg; filren-0@student.ltu.se

We are gonna do a simple lab where we train a model using Keras

We are gonna use the mnist dataset of digits imported from keras like we did in lab 1 and lab 4.

In [1]:
from silence_tensorflow import silence_tensorflow
silence_tensorflow() # Remove non-important tensorflow warnings

import numpy as np
import keras
from keras import layers
from keras.datasets import mnist
input_shape = (28,28,1)
num_classes = 10

We preprocess our data

In [2]:
(train_data, train_labels), (test_data, test_labels) = mnist.load_data()
#normalize input values from RGB to Greyscale [0,1]
train_data = train_data.astype("float32")/255
test_data = test_data.astype("float32")/255
#Make sure images have the correct shape(28,28,1)
train_data = np.expand_dims(train_data, -1)
test_data = np.expand_dims(test_data, -1)

#converts class vectors to binary class matrices
train_labels = keras.utils.to_categorical(train_labels, num_classes)
test_labels = keras.utils.to_categorical(test_labels, num_classes)

print("training shape: ", train_data.shape)
print(train_data.shape[0], "number of training samples")
print(test_data.shape[0], "number of testing samples")

training shape:  (60000, 28, 28, 1)
60000 number of training samples
10000 number of testing samples


We create our multi-layer neural network using Keras

In [28]:
def create_model(nr_conv2d = 1, nr_filter = 32, kernel_size = (3, 3)):
    # Name the model based on the input values, 
    # model_nr_conv2d-nr_filter-kernel_size
    model_name=f"model_{nr_conv2d}-{nr_filter}-{kernel_size[0]}"
    
    hidden_layers = []

    hidden_layers.append(keras.Input(shape=input_shape)) # input layer

    for i in range(nr_conv2d):
        hidden_layers.append(layers.Conv2D(nr_filter, kernel_size=kernel_size, activation="relu"))
        hidden_layers.append(layers.MaxPooling2D(pool_size=[2,2]))

        nr_filter*=2

    hidden_layers.append(layers.Flatten())
    hidden_layers.append(layers.Dropout(0.5)) #prevents overfitting
    hidden_layers.append(layers.Dense(num_classes, activation="softmax"))

    model = keras.Sequential(hidden_layers, name=model_name)

    model.summary()
    
    return model

Next we are training the model. We can use several different loss functions here, for this project we will use "categorical_crossentropy", "sparse_categorical_crossentropy" and "binary_crossentropy"

In [4]:
n_epochs = 10
size_batch = 128

def model_training(pick, model):
    match pick:
        case 1:
            #categorical loss function
            model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
            print("Categorical loss function")
            model.fit(train_data, train_labels, batch_size=size_batch, epochs=n_epochs, validation_split=0.1, verbose=0)
        case 2:
            #poisson loss function
            model.compile(loss="poisson", optimizer="adam", metrics=["accuracy"])
            print("Poisson loss function")
            model.fit(train_data, train_labels, batch_size=size_batch, epochs=n_epochs, validation_split=0.1, verbose=0)
        case 3:
            #binary loss function
            model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
            print("Binary loss function")
            model.fit(train_data, train_labels, batch_size=size_batch, epochs=n_epochs, validation_split=0.1, verbose=0)


Define for the test diffrent values we want to test

In [5]:
nr_conv2d_list = [1, 2, 3]
nr_filter_list = [16, 32, 64]

# Boxes to save accuracy and loss values
# 1 the layer, filter or loss fucntion value; 2 the accuracy; 3 the loss value
nr_layer_values = []
nr_filter_values = []
loss_function_values = []
best_model = [[0, 1]]

In [16]:
def best_performance(best_model):
    # Resulting best model:
    create_model(nr_conv2d=best_model[0][2], nr_filter=best_model[0][3])
    match best_model[0][4]:
        case 1:
            print("Using Categorical crossentropy loss function it gave us the \nfollowing accuracy and loss:")
        case 2:
            print("Using Poisson loss function it gave us the \nfollowing accuracy and loss:")
        case 3:
            print("Using binary crossentropy loss function it gave us the \nfollowing accuracy and loss")
    print("|\taccuracy: \t", best_model[0][0], "\t|")
    print("|\tloss: \t\t", best_model[0][1], "\t|")

In [6]:
for i in nr_conv2d_list: # The nr conv2d layers
    for j in nr_filter_list: # The starting size of the conv2d filters
        for n in range(3):

            model = create_model(nr_conv2d=i, nr_filter=j, kernel_size=(3, 3)) # You can also change kernel size
            train = model_training(n+1, model)

            #evaluate
            score = model.evaluate(train_data, train_labels, verbose=0)
            print("accuracy: \t", score[1])
            print("loss: \t\t", score[0])

            nr_layer_values.append((i, score[1], score[0]))
            nr_filter_values.append((j, score[1], score[0]))
            loss_function_values.append((n+1, score[1], score[0]))

            if (score[1] > best_model[0][0]) & (score[0] < best_model[0][1]):
                best_model.pop()
                best_model.append((score[1], score[0], i, j, n+1))

            print("\n")

print("Finally done :)")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 16)        160       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 16)        0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 2704)              0         
                                                                 
 dropout (Dropout)           (None, 2704)              0         
                                                                 
 dense (Dense)               (None, 10)                27050     
                                                                 
Total params: 27210 (106.29 KB)
Trainable params: 27210 (106.29 KB)
Non-trainable params: 0 (0.00 Byte)
__________________

The following is the best performing model from our tests:

In [29]:
best_performance(best_model)

Model: "model_3-64-3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_81 (Conv2D)          (None, 26, 26, 64)        640       
                                                                 
 max_pooling2d_81 (MaxPooli  (None, 13, 13, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_82 (Conv2D)          (None, 11, 11, 128)       73856     
                                                                 
 max_pooling2d_82 (MaxPooli  (None, 5, 5, 128)         0         
 ng2D)                                                           
                                                                 
 conv2d_83 (Conv2D)          (None, 3, 3, 256)         295168    
                                                                 
 max_pooling2d_83 (MaxPooli  (None, 1, 1, 256)        